## Data Acquisition - Crawling 

In [1]:
# import re for reges
import re

import time
import datetime
from bs4 import BeautifulSoup
import requests
import json
import pandas as pd


# Import webdriver to initialise a browser
from selenium import webdriver
from selenium.webdriver import ActionChains
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support import expected_conditions as EC

### definning functions to generate start and end dates of months.

In [2]:
### code ref: https://stackoverflow.com/a/13565185/8703160
def end_of_month(any_day):
    # this will never fail
    # get close to the end of the month for any day, and add 4 days 'over'
    next_month = any_day.replace(day=28) + datetime.timedelta(days=4)
    # subtract the number of remaining 'overage' days to get last day of current month, or said programattically said, the previous day of the first of next month
    return next_month - datetime.timedelta(days=next_month.day)

def end_date_of_month_gen():
    end_arr = []
    for month in range(1, 13):
        for year in range (2020, 2022):
            end_date = (end_of_month(datetime.date(year, month, 1)))
            end_arr.append(str(end_date))
    return end_arr



In [3]:
def start_day_of_month():
    start_arr = []
    for month in range (1,13):
        for year in range(2020,2022):
            start_day_of_month = datetime.date(year, month, 1)
            start_arr.append(str(start_day_of_month))
    return start_arr





### Login function with selenium for the MSW website using my credentials, I have deleted the email and password for privacy and security.

In [4]:
#login
def login(self):
    email = self.find_element(By.NAME, 'email')
    password = self.find_element(By.NAME, 'password')
    email.send_keys("")
    password.send_keys("")
    self.find_element(By.ID,"msw-js-login").click()
    time.sleep(5)
    



### Start web driver session  with selenium

In [5]:
baseUrl = "https://magicseaweed.com/account/login/?dest=%2FRishon-Lezion-Surf-Report%2F3976%2FHistoric%2F%3Fstart%3D2020-01-01%26end%3D2020-01-05"
# Initialize webdriver and put the path where download the driver
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
# Launch Chrome and pass the url
driver.get(baseUrl)
login(driver)





====== WebDriver manager ======
Current google-chrome version is 97.0.4692
Get LATEST chromedriver version for 97.0.4692 google-chrome
Driver [C:\Users\Assaf\.wdm\drivers\chromedriver\win32\97.0.4692.71\chromedriver.exe] found in cache


### store the dates in arrays

In [6]:
start_date = start_day_of_month()
print(start_date)
end_date = end_date_of_month_gen()
print(end_date)

['2020-01-01', '2021-01-01', '2020-02-01', '2021-02-01', '2020-03-01', '2021-03-01', '2020-04-01', '2021-04-01', '2020-05-01', '2021-05-01', '2020-06-01', '2021-06-01', '2020-07-01', '2021-07-01', '2020-08-01', '2021-08-01', '2020-09-01', '2021-09-01', '2020-10-01', '2021-10-01', '2020-11-01', '2021-11-01', '2020-12-01', '2021-12-01']
['2020-01-31', '2021-01-31', '2020-02-29', '2021-02-28', '2020-03-31', '2021-03-31', '2020-04-30', '2021-04-30', '2020-05-31', '2021-05-31', '2020-06-30', '2021-06-30', '2020-07-31', '2021-07-31', '2020-08-31', '2021-08-31', '2020-09-30', '2021-09-30', '2020-10-31', '2021-10-31', '2020-11-30', '2021-11-30', '2020-12-31', '2021-12-31']


### definning the scraping data function using crawling with beautiful-soup

In [7]:
def scrape_data(soup):
    date = []
    # hours = ['00:00', '03:00', '06:00', '09:00', '12:00', '15:00', '18:00','21:00']
    hours = []
    primarySwellHeight = []
    primarySwellPeriod = []
    secondarySwellHeight = []
    secondarySwellPeriod = []
    swellRating = []

    table  = soup.find('table', {'class':'table table-primary table-forecast allSwellsActive msw-js-table msw-units-large'})
    for tbody in table.find_all('tbody'):
        for tr in tbody.find_all('tr'):
            if tr.has_attr('data-timestamp'):
                date.append(time.strftime('%Y-%m-%d %H:%M', time.localtime(int(tr['data-timestamp']))))
            for td_time in tr.find_all('td', {'class':'nopadding-left row-title background-clear msw-js-tooltip'}):
                hours.append(td_time.find('small').get_text().strip())

                
            for i, td_swellHeight in enumerate(tr.find_all('td', {'class':['text-center background-gray-lighter','text-center background-gray-lightest']})):
                if i % 2 == 0:    
                    for h4_info in td_swellHeight.find_all('h4'):
                        primarySwellHeight.append(h4_info.text.strip())
                elif i % 2 != 0:
                    for h4_info in td_swellHeight.find_all('h4'):
                        primarySwellPeriod.append(h4_info.text.strip())
                else:
                    continue
                        

            count = 0
            for td_rate in tr.find_all('ul'):
                count = len(td_rate.find_all('li', {'class':'active'}))
                swellRating.append(count)
    return pd.DataFrame({'date': pd.Series(date),'hour': pd.Series(hours), 'swell_rating': pd.Series(swellRating), 'primary_swell_height': pd.Series(primarySwellHeight), 'primary_swell_period': pd.Series(primarySwellPeriod)})


            




### here we iteratively insert the start date and end date in the url so the url will change for each month and send it to the scape data function to get all the data that we need we store all the scraped data in list L

In [8]:

def get_res_for_url(start, end):
    r = []
    dynamicUrl = ["https://www.magicseaweed.com/Rishon-Lezion-Surf-Report/3976/Historic/", "https://www.magicseaweed.com/Bat-Yam-Surf-Report/3662/Historic/", "https://www.magicseaweed.com/Hilton-Surf-Report/3658/Historic/"]
    for url in dynamicUrl:
        url += "?start=" + start + "&amp;end=" + end + "&end=" + end
        driver.get(url)    
        r.append(driver.page_source)
    return r

L = []

for i,j in zip(start_date, end_date):
    html = get_res_for_url(i,j)
    for h in html:
        soup = BeautifulSoup(h, "html.parser")
        time.sleep(3)
        L.append(scrape_data(soup))



### I merged the data in L using concat 

In [9]:
df = pd.concat(L, ignore_index = True)

### save the dataframe as csv file

In [10]:
df.to_csv('info.csv')

In [11]:
df

,date,hour,swell_rating,primary_swell_height,primary_swell_period
0,2020-01-01 02:00,12am,1,1m,8s
1,2020-01-01 05:00,3am,0,1m,7s
2,2020-01-01 08:00,6am,1,0.9m,7s
3,2020-01-01 11:00,9am,1,0.8m,7s
4,2020-01-01 14:00,12pm,0,0.8m,7s
...,...,...,...,...,...
17538,2021-12-31 09:00,9am,1,1.1m,7s
17539,2021-12-31 12:00,12pm,1,1m,8s
17540,2021-12-31 15:00,3pm,1,0.9m,8s
17541,2021-12-31 18:00,6pm,1,0.9m,8s
